In [18]:
import os
import numpy as np
import vgg16
import data_parser

ImportError: No module named termcolor

In [ ]:
model=vgg16()
print("loaded model")

In [ ]:
train_data = data_parser()
model.setup_training(session)

opt = tf.train.AdamOptimizer(self.cfgs['optimizer_params']['learning_rate'])
train = opt.minimize(self.model.loss)

session.run(tf.global_variables_initializer())

for idx in range(self.cfgs['max_iterations']):

    im, em, _ = train_data.get_training_batch()

    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()

    _, summary, loss = session.run([train, model.merged_summary, model.loss],
                                   feed_dict={model.images: im, model.edgemaps: em},
                                   options=run_options,
                                   run_metadata=run_metadata)

    model.train_writer.add_run_metadata(run_metadata, 'step{:06}'.format(idx))
    model.train_writer.add_summary(summary, idx)

    print('[{}/{}] TRAINING loss : {}'.format(idx, self.cfgs['max_iterations'], loss))

    if idx % self.cfgs['save_interval'] == 0:

        saver = tf.train.Saver()
        saver.save(session, os.path.join(self.cfgs['save_dir'], 'models/hed-model'), global_step=idx)

    if idx % self.cfgs['val_interval'] == 0:

        im, em, _ = train_data.get_validation_batch()

        summary, error = session.run([self.model.merged_summary, self.model.error], feed_dict={self.model.images: im, self.model.edgemaps: em})

        self.model.val_writer.add_summary(summary, idx)
        self.io.print_info('[{}/{}] VALIDATION error : {}'.format(idx, self.cfgs['max_iterations'], error))

self.model.train_writer.close()